In [1]:
#importing necessary libraries.
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Functions

# Function to extract Articles Headlines
def headlines(soupsss):
    try:
        title = soupsss.find("h1", class_="ar__ti")
    # Ensure the tag was found
        if title:
        # Extract and clean the text
            title_value = title.get_text(strip=True)
            return title_value
        else:
            return "Title not found"
    except Exception as e:
        return f"An error occurred: {e}"
#print(headlines(soupsss))

# Function to extract Publishing date
def date(soupsss):
    try:
        publish_date = soupsss.find("time", class_="published updated meta-data d-inline mt-2 font-sans black-70")
    # Ensure the tag was found
        if publish_date:
        # Extract and clean the text
            publish_date_value = publish_date.get_text(strip=True)
            return publish_date_value
        else:
            return "Title not found"
    except Exception as e:
        return f"An error occurred: {e}"
#print(date(soupsss))

# Function to extract Articles' writer 
def author_name(soupsss):
    try:
        author = soupsss.find("a", class_="byline__author__link url fn text-decoration-none black-70")
    # Ensure the tag was found
        if author:
        # Extract and clean the text
            author_value = author.get_text(strip=True)
            return author_value
        else:
            return "Aurhor not found"
    except Exception as e:
        return f"An error occurred: {e}"
#print(author_name(soupsss))

def articles(soupsss):
    paragraphs = ''
    # Find all elements in the order they appear
    for tag in soupsss.find_all(True):  # `True` finds all tags
        # Stop processing if the <div> with class "truth-post-content-after" is encountered
        if tag.name == "div" and tag.get("class") == ["truth-post-content-after"]:
            break
        # If the tag is a <p>, extract its text
        if tag.name == "p":
            paragraphs=paragraphs+ tag.get_text(strip=True)
    
    return paragraphs
#print(articles(soupsss))

In [3]:
# Links
current_page = 1

link_data = []

proceed = True

while(proceed):
    #print("Magic is happening on the page number: "+str(current_page))
    url = "https://truthout.org/topics/donald-trump/page/"+str(current_page)+"/"
    
    # Headers for request
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}) 

    page = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(page.text,"html.parser")

    if soup.title.text == "Page not found | Truthout":
        proceed = False
    
    else:
        links = soup.find_all("h2", attrs={'class':"ar-list__ti mb-3 mb-md-2"})
        hrefs = [h2.find("a")["href"] for h2 in links]
        link_data.append([h2.find("a")["href"] for h2 in links])

    current_page += 1

In [4]:
# getting link list are inside of list
final_links_list = [item for sublist in link_data for item in sublist]

In [6]:
len(final_links_list)

6813

In [ ]:
d = {"Headlines":[], "Publish_date":[], "Writers":[], "Articles":[]}
    
# Loop for extracting articles details from each link 
for link in final_links_list:
    new_webpage = requests.get(link, headers=HEADERS)

    new_soup = BeautifulSoup(new_webpage.content, "html.parser")

    # Function calls to display all necessary articles information
    d['Headlines'].append(headlines(new_soup))
    d['Publish_date'].append(date(new_soup))
    d['Writers'].append(author_name(new_soup))
    d['Articles'].append(articles(new_soup))

In [8]:
df= pd.DataFrame(d)
df.to_csv("truthout1.csv", header=True, index=False) #if you wanna save as .csv file